Analytical Base Table (ABT) is a flat table that is used for building analytical models and scoring (predicting) the future behavior of a subject.

# Cryptocurrency casestudy 

<hr>

For example,. let’s say we wanted to run a step-forward analysis of a very rudimentary momentum trading strategy that goes as follows:

1. At the start of every month, we buy the cryptocurrency that had the largest price gain over the previous 7, 14, 21, or 28 days. We want to evaluate each of these time windows.
2. Then, we hold for exactly 7 days and sell our position. Please note: this is a purposefully simple strategy that is only meant for illustrative purposes.

How well would we go about evaluating this strategy?

This is a great question for showcasing data wrangling techniques because all the hard work lies in molding your dataset into the proper format. Once you have the appropriate analytical base table (ABT), answering the question becomes simple.

<hr>

In [3]:
# Import libraries and dataset
import pandas as pd
import numpy as np

flatten = lambda l: [item for sublist in l for item in sublist]

pd.options.display.float_format = '{:,.2f}'.format

flatten= lambda l: [item for sublist in l for item in sublist]
data_path = r'C:\Users\emmam\Documents\nlb\data\BNC2_sample.csv'

In [4]:
df = pd.read_csv(data_path)

* data: BNC2_sample.csv

headers: ['Code', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'TWAP']

In [5]:
# Please read the csv file and give the headers as shown above
df0_colnames = ['Code', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'TWAP']
df = pd.read_csv(data_path, header=None, names=df0_colnames)

df

,Code,Date,Open,High,Low,Close,Volume,VWAP,TWAP
0,GWA_BTC,2014-04-01,467.28,488.62,467.28,479.56,"74,776.48",482.76,482.82
1,GWA_BTC,2014-04-02,479.20,494.30,431.32,437.08,"114,052.96",460.19,465.93
2,GWA_BTC,2014-04-03,437.33,449.74,414.41,445.60,"91,415.08",432.29,433.28
3,GWA_BTC,2014-04-04,445.18,456.10,429.16,449.81,"51,147.27",443.46,443.93
4,GWA_BTC,2014-04-05,450.08,464.09,445.16,461.70,"28,449.19",452.53,452.95
...,...,...,...,...,...,...,...,...,...
31756,MWA_XRP_USD,2018-01-19,1.57,1.69,1.48,1.53,"242,563,870.44",1.59,1.59
31757,MWA_XRP_USD,2018-01-20,1.54,1.62,1.49,1.57,"140,459,727.30",1.56,1.56
31758,MWA_XRP_USD,2018-01-21,1.57,1.57,1.32,1.36,"219,345,506.61",1.42,1.43
31759,MWA_XRP_USD,2018-01-22,1.37,1.42,1.15,1.35,"276,793,376.41",1.30,1.31


In [6]:
# Data Dictionary (for code GWA_BTC):

# Date: The day on which the index values were calculated.
# Open: The day's opening price index for Bitcoin in US dollars.
# High: The highest value for the price index for Bitcoin in US dollars that day.
# Low: The lowest value for the price index for Bitcoin in US dollars that day.
# Close: The day's closing price index for Bitcoin in US dollars.
# Volume: The volume of Bitcoin traded that day.
# VWAP: The volume weighted average price of Bitcoin traded that day.
# TWAP: The time-weighted average price of Bitcoin traded that day. 

# (Open + High + Low + Close)/4


# Understand the data (Research on the domain knowledge)

One of the most common reasons to wrangle data is when there's "too much" information packed into a single table, especially when dealing with time series data.

Generally, all observations should be equivalent in granularity and in units.

There will be exceptions, but for the most part, this rule of thumb can save you from many headaches.

* Equivalence in Granularity - For example, you could have 10 rows of data from 10 different cryptocurrencies. However, you should not have an 11th row with average or total values from the other 10 rows. That 11th row would be an aggregation, and thus not equivalent in granularity to the other 10.

* Equivalence in Units - You could have 10 rows with prices in USD collected at different dates. However, you should not then have another 10 rows with prices quoted in EUR. Any aggregations, distributions, visualizations, or statistics would become meaningless.

Our current raw dataset breaks both of these rules!

In [7]:
df.Code.unique()

array(['GWA_BTC', 'GWA_ETH', 'GWA_LTC', 'GWA_XLM', 'GWA_XRP',
       'MWA_BTC_CNY', 'MWA_BTC_EUR', 'MWA_BTC_GBP', 'MWA_BTC_JPY',
       'MWA_BTC_USD', 'MWA_ETH_CNY', 'MWA_ETH_EUR', 'MWA_ETH_GBP',
       'MWA_ETH_JPY', 'MWA_ETH_USD', 'MWA_LTC_CNY', 'MWA_LTC_EUR',
       'MWA_LTC_GBP', 'MWA_LTC_JPY', 'MWA_LTC_USD', 'MWA_XLM_CNY',
       'MWA_XLM_EUR', 'MWA_XLM_USD', 'MWA_XRP_CNY', 'MWA_XRP_EUR',
       'MWA_XRP_GBP', 'MWA_XRP_JPY', 'MWA_XRP_USD'], dtype=object)

GWA and MWA? These are actually completely different types of indicators according to the documentation page: https://www.quandl.com/data/BNC2-BNC-Digital-Currency-Indexed-EOD/documentation

* MWA stands for "market-weighted average," and they show regional prices. There are multiple MWA codes for each cryptocurrency, one for each local fiat currency.
* On the other hand, GWA stands for "global-weighted average," which shows globally indexed prices. GWA is thus an aggregation of MWA and not equivalent in granularity. (Note: only a subset of regional MWA codes are included in the sample dataset.)


In [12]:
# Check for youself
# show GWA and serveral MWA values for bitcoins on Jan 1, 2018:
# // your code here

display(
df[(df['Date'] == '2018-01-01') & (df['Code'].str.startswith('GWA')) | (df['Code'].str.startswith('MWA'))  ].sample(5),
df[(df['Date'] == '2018-01-01') & (df['Code'].str.startswith('GWA')) ],
df[(df['Date'] == '2018-01-01') & (df['Code'].str.startswith('MWA')) ].sample(5),
    )


,Code,Date,Open,High,Low,Close,Volume,VWAP,TWAP
24919,MWA_XLM_EUR,2017-05-13,0.04,0.04,0.03,0.03,"28,241,840.11",0.04,0.04
8857,MWA_BTC_EUR,2017-05-29,"1,905.43","2,092.61","1,868.47","2,041.79","33,753.14","1,987.84","1,986.27"
9634,MWA_BTC_GBP,2015-09-20,154.41,155.39,150.66,151.51,"1,687.88",152.70,152.86
11772,MWA_BTC_JPY,2017-10-27,"667,991.54","676,654.80","644,654.29","651,449.87","112,091.49","660,032.80","659,789.61"
16876,MWA_ETH_USD,2016-04-16,8.25,8.59,8.16,8.58,"38,891.85",8.29,8.29


,Code,Date,Open,High,Low,Close,Volume,VWAP,TWAP
1371,GWA_BTC,2018-01-01,"14,505.89","14,505.89","13,617.46","14,092.74","225,906.21","14,103.18","14,093.73"
2271,GWA_ETH,2018-01-01,758.46,785.58,742.40,774.82,"2,061,583.32",765.22,765.15
3665,GWA_LTC,2018-01-01,231.05,235.84,221.05,228.39,"2,578,133.63",227.89,227.91
4892,GWA_XLM,2018-01-01,0.36,0.53,0.36,0.48,"1,452,511,305.98",0.45,0.44
6286,GWA_XRP,2018-01-01,2.25,2.35,2.09,2.35,"813,772,647.20",2.20,2.20


,Code,Date,Open,High,Low,Close,Volume,VWAP,TWAP
30388,MWA_XRP_JPY,2018-01-01,244.14,247.73,221.60,247.73,"6,042,499.20",237.98,235.90
22268,MWA_LTC_JPY,2018-01-01,"27,494.33","28,753.61","26,426.24","26,454.29","1,187.81","27,856.58","27,635.65"
26835,MWA_XRP_CNY,2018-01-01,14.78,14.86,13.59,14.86,"2,630,602.58",14.32,14.27
21595,MWA_LTC_GBP,2018-01-01,207.19,233.72,190.21,191.85,104.26,205.20,204.47
14981,MWA_ETH_EUR,2018-01-01,638.25,649.42,623.92,641.38,"58,181.51",637.08,637.06


In [13]:
# Filter unwanted observations (keep only GWA)
# // your code here
df = df.loc[df['Code'].str.contains('GWA')] 
df['Date'] = df['Date'].apply(pd.to_datetime)

display(
    df['Date'].sample(15),
    df['Date'].isna().sum(),
)


<ipython-input-13-6a91241871a0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date']= df['Date'].apply(pd.to_datetime)


2079   2017-06-23
1859   2016-11-15
1850   2016-11-06
2023   2017-04-28
2166   2017-09-18
6230   2017-11-06
289    2015-01-15
3309   2017-01-10
5355   2015-06-15
3151   2016-08-05
677    2016-02-07
3174   2016-08-28
1272   2017-09-24
5471   2015-10-09
1670   2016-05-10
Name: Date, dtype: datetime64[ns]

0

In [14]:
df.Code.unique()

array(['GWA_BTC', 'GWA_ETH', 'GWA_LTC', 'GWA_XLM', 'GWA_XRP'],
      dtype=object)

In [15]:
coin_keys = df.Code.unique()

#coin_df_lst = [df.Code == cc for cc in coin_keys]
#coin_df_dict = {cc[-3:] : df[df.Code == cc] for cc in coin_keys}
#display(coin_df_dict.items(), coin_df_dict['ETH'].shape)


In [16]:
#display('split coins into sub dframes -- ', [df_s for df_s in coin_df_lst] )

cols_keys = list(df.columns)

# #cols_keys.remove('Code') , cols_keys.remove('Date')

display(coin_keys, cols_keys)
# coin_df_lst = list()
#for c_k in coin_keys:
#     df_suffix = c_k[-3:] # like BTC, LTC...
#     {c_k[-3:]}
#     exec(f'df_{c_k[-3:]} = df[df.Code=="{c_k}"]')
#     exec(f'display(df_{c_k[-3:]})')
#     coin_df_lst.append(f'df_{c_k[-3:]}')

array(['GWA_BTC', 'GWA_ETH', 'GWA_LTC', 'GWA_XLM', 'GWA_XRP'],
      dtype=object)

['Code', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP', 'TWAP']

In [17]:
cols_keys.remove("Date")

In [27]:
# Pivot the dataset

cols_keys1=[str(s) for s in df.columns if s not in ['Date']]

pivoted_df = df.pivot(index="Date", columns='Code',
                      values = "VWAP", 
                     )
pivoted_df

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
2014-04-01,482.76,nan,13.19,nan,0.01
2014-04-02,460.19,nan,12.09,nan,0.01
2014-04-03,432.29,nan,10.14,nan,0.01
2014-04-04,443.46,nan,10.95,nan,0.01
2014-04-05,452.53,nan,11.03,nan,0.01
...,...,...,...,...,...
2018-01-19,"11,826.36","1,068.45",195.00,0.51,1.82
2018-01-20,"13,062.68","1,158.71",207.58,0.52,1.75
2018-01-21,"12,326.23","1,108.90",197.36,0.48,1.55


In [42]:
# 6. Shift the pivoted dataset

# Following is a small test to see if we shift down 1 row for the last three rows, what it looks like

display(
'expr a',
pivoted_df.tail(3).shift(1)['GWA_ETH'],
# 'expr b',
# pivoted_df['VWAP'].tail(3).shift(1),
)


display(
    pivoted_df.info(),
    #pivoted_df['Code'],
    pivoted_df,
# 'expr a',
# pivoted_df.tail(5).shift(3)['VWAP'],
# 'expr b',
# pivoted_df['VWAP'].tail(5).shift(3),
)

'expr a'

Date
2018-01-21        nan
2018-01-22   1,108.90
2018-01-23   1,038.21
Name: GWA_ETH, dtype: float64

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1394 entries, 2014-04-01 to 2018-01-23
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   GWA_BTC  1394 non-null   float64
 1   GWA_ETH  900 non-null    float64
 2   GWA_LTC  1394 non-null   float64
 3   GWA_XLM  1227 non-null   float64
 4   GWA_XRP  1394 non-null   float64
dtypes: float64(5)
memory usage: 65.3 KB


None

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
2014-04-01,482.76,nan,13.19,nan,0.01
2014-04-02,460.19,nan,12.09,nan,0.01
2014-04-03,432.29,nan,10.14,nan,0.01
2014-04-04,443.46,nan,10.95,nan,0.01
2014-04-05,452.53,nan,11.03,nan,0.01
...,...,...,...,...,...
2018-01-19,"11,826.36","1,068.45",195.00,0.51,1.82
2018-01-20,"13,062.68","1,158.71",207.58,0.52,1.75
2018-01-21,"12,326.23","1,108.90",197.36,0.48,1.55


In [30]:
pivoted_df.tail(3).shift(1)

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
2018-01-21,nan,nan,nan,nan,nan
2018-01-22,"12,326.23","1,108.90",197.36,0.48,1.55
2018-01-23,"11,397.52","1,038.21",184.92,0.47,1.43


In [37]:
'''
"(percent of) return over a period of time"

R = (Vf - Vi)/Vi = Vf/Vi - 1
'''


#demo_return_1 = df1.tail(3)/df1.tail(3).shift(1) -1



#def return_over_time(df)

#display(pivoted_df['Date'])



test = pivoted_df.tail(3)/pivoted_df.tail(3).shift(1) -1


test

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
2018-01-21,nan,nan,nan,nan,nan
2018-01-22,-0.08,-0.06,-0.06,-0.03,-0.08
2018-01-23,-0.04,-0.04,-0.04,0.01,-0.00


Notice how the shifted dataset now has values from 1 day before? We can take advantage of this to calculate prior returns for our 7, 14, 21, 28 day windows.

For example, to calculate returns over the 7 days prior, we would need  prices_today / prices_7_days_ago - 1.0, which translates to:



In [34]:

# Calculate returns over 7 days prior
delta_7 = pivoted_df / pivoted_df.shift(7) - 1.0
 
# Display examples
delta_7.tail()

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
2018-01-19,-0.18,-0.17,-0.18,-0.21,-0.22
2018-01-20,-0.13,-0.19,-0.18,-0.23,-0.29
2018-01-21,-0.15,-0.20,-0.22,-0.22,-0.30
2018-01-22,-0.21,-0.24,-0.24,-0.25,-0.32
2018-01-23,-0.11,-0.12,-0.13,-0.02,-0.04


In [44]:
# Same thing for 7 days, 14 days, 21 days, 28 days. 
# generate a dictionary to hold all these dataframes with the key equals to the column name 'delata_7', 'delta_14', etc..
delta_dict = {}

#  // your code here
offset_lst=[7, 14, 21, 28]
for offset in offset_lst:
    delta_dict[f'delta_{offset}']= pivoted_df/pivoted_df.shift(offset)-1
    

delta_dict

{'delta_7': Code        GWA_BTC  GWA_ETH  GWA_LTC  GWA_XLM  GWA_XRP
 Date                                                   
 2014-04-01      nan      nan      nan      nan      nan
 2014-04-02      nan      nan      nan      nan      nan
 2014-04-03      nan      nan      nan      nan      nan
 2014-04-04      nan      nan      nan      nan      nan
 2014-04-05      nan      nan      nan      nan      nan
 ...             ...      ...      ...      ...      ...
 2018-01-19    -0.18    -0.17    -0.18    -0.21    -0.22
 2018-01-20    -0.13    -0.19    -0.18    -0.23    -0.29
 2018-01-21    -0.15    -0.20    -0.22    -0.22    -0.30
 2018-01-22    -0.21    -0.24    -0.24    -0.25    -0.32
 2018-01-23    -0.11    -0.12    -0.13    -0.02    -0.04
 
 [1394 rows x 5 columns],
 'delta_14': Code        GWA_BTC  GWA_ETH  GWA_LTC  GWA_XLM  GWA_XRP
 Date                                                   
 2014-04-01      nan      nan      nan      nan      nan
 2014-04-02      nan      nan      na

Calculating returns by shifting the dataset requires 2 assumptions to be met: (1) the observations are sorted ascending by date and (2) there are no missing dates. We checked this "off-stage" to keep this tutorial concise, but we recommend confirming this on your own.

## Melt the shifted dataset

Now that we've calculated returns using the pivoted dataset, we're going to "unpivot" the returns. By unpivoting, or melting the data, we can later create an analytical base table (ABT) where each row contains all of the relevant information for a particular coin on a particular date.

We couldn't directly shift the original dataset because the data for different coins were stacked on each other, so the boundaries would've overlapped. In other words, BTC data would leak into ETH calculations, ETH data would leak into LTC calculations, and so on.

To melt the data, we'll...

* reset_index() so we can call the columns by name.
* Call the  melt() method.
* Pass the column(s) to keep into the  id_vars= argument.
* Name the melted column using the  value_name= argument.

Here's how that looks for one dataframe:

In [45]:
delta_7.head(10)

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
2014-04-01,nan,nan,nan,nan,nan
2014-04-02,nan,nan,nan,nan,nan
2014-04-03,nan,nan,nan,nan,nan
2014-04-04,nan,nan,nan,nan,nan
2014-04-05,nan,nan,nan,nan,nan
2014-04-06,nan,nan,nan,nan,nan
2014-04-07,nan,nan,nan,nan,nan
2014-04-08,-0.06,nan,-0.14,nan,-0.08
2014-04-09,-0.02,nan,-0.06,nan,0.05


In [46]:
delta_7.tail(10)

Code,GWA_BTC,GWA_ETH,GWA_LTC,GWA_XLM,GWA_XRP
Date,,,,,
2018-01-14,-0.17,0.24,-0.13,-0.11,-0.31
2018-01-15,-0.09,0.14,-0.08,-0.03,-0.29
2018-01-16,-0.22,-0.11,-0.21,-0.22,-0.46
2018-01-17,-0.29,-0.31,-0.30,-0.27,-0.47
2018-01-18,-0.19,-0.18,-0.19,-0.04,-0.22
2018-01-19,-0.18,-0.17,-0.18,-0.21,-0.22
2018-01-20,-0.13,-0.19,-0.18,-0.23,-0.29
2018-01-21,-0.15,-0.20,-0.22,-0.22,-0.30
2018-01-22,-0.21,-0.24,-0.24,-0.25,-0.32


In [47]:
# Melt delta_7 returns
melted_7 = delta_7.reset_index().melt(id_vars=['Date'], value_name='delta_7')
 
# Melted dataframe examples
melted_7

,Date,Code,delta_7
0,2014-04-01,GWA_BTC,nan
1,2014-04-02,GWA_BTC,nan
2,2014-04-03,GWA_BTC,nan
3,2014-04-04,GWA_BTC,nan
4,2014-04-05,GWA_BTC,nan
...,...,...,...
6965,2018-01-19,GWA_XRP,-0.22
6966,2018-01-20,GWA_XRP,-0.29
6967,2018-01-21,GWA_XRP,-0.30
6968,2018-01-22,GWA_XRP,-0.32


In [52]:
# Now it's your turn to melt the shifted dataset:
# Melt all the delta dataframes and store in list
melted_dfs = []
# // your code here

for offset in offset_lst:
    melted_dfs.append(
    delta_dict[f'delta_{offset}'].reset_index().melt(id_vars= ['Date'], 
                                                       value_name = f"delta_{offset}")
    )
melted_dfs[:]

[           Date     Code  delta_7
 0    2014-04-01  GWA_BTC      nan
 1    2014-04-02  GWA_BTC      nan
 2    2014-04-03  GWA_BTC      nan
 3    2014-04-04  GWA_BTC      nan
 4    2014-04-05  GWA_BTC      nan
 ...         ...      ...      ...
 6965 2018-01-19  GWA_XRP    -0.22
 6966 2018-01-20  GWA_XRP    -0.29
 6967 2018-01-21  GWA_XRP    -0.30
 6968 2018-01-22  GWA_XRP    -0.32
 6969 2018-01-23  GWA_XRP    -0.04
 
 [6970 rows x 3 columns],
            Date     Code  delta_14
 0    2014-04-01  GWA_BTC       nan
 1    2014-04-02  GWA_BTC       nan
 2    2014-04-03  GWA_BTC       nan
 3    2014-04-04  GWA_BTC       nan
 4    2014-04-05  GWA_BTC       nan
 ...         ...      ...       ...
 6965 2018-01-19  GWA_XRP     -0.41
 6966 2018-01-20  GWA_XRP     -0.42
 6967 2018-01-21  GWA_XRP     -0.51
 6968 2018-01-22  GWA_XRP     -0.52
 6969 2018-01-23  GWA_XRP     -0.48
 
 [6970 rows x 3 columns],
            Date     Code  delta_21
 0    2014-04-01  GWA_BTC       nan
 1    2014-04-02  GW

In [53]:
print(melted_dfs)

[           Date     Code  delta_7
0    2014-04-01  GWA_BTC      nan
1    2014-04-02  GWA_BTC      nan
2    2014-04-03  GWA_BTC      nan
3    2014-04-04  GWA_BTC      nan
4    2014-04-05  GWA_BTC      nan
...         ...      ...      ...
6965 2018-01-19  GWA_XRP    -0.22
6966 2018-01-20  GWA_XRP    -0.29
6967 2018-01-21  GWA_XRP    -0.30
6968 2018-01-22  GWA_XRP    -0.32
6969 2018-01-23  GWA_XRP    -0.04

[6970 rows x 3 columns],            Date     Code  delta_14
0    2014-04-01  GWA_BTC       nan
1    2014-04-02  GWA_BTC       nan
2    2014-04-03  GWA_BTC       nan
3    2014-04-04  GWA_BTC       nan
4    2014-04-05  GWA_BTC       nan
...         ...      ...       ...
6965 2018-01-19  GWA_XRP     -0.41
6966 2018-01-20  GWA_XRP     -0.42
6967 2018-01-21  GWA_XRP     -0.51
6968 2018-01-22  GWA_XRP     -0.52
6969 2018-01-23  GWA_XRP     -0.48

[6970 rows x 3 columns],            Date     Code  delta_21
0    2014-04-01  GWA_BTC       nan
1    2014-04-02  GWA_BTC       nan
2    2014-04-0

Finally, we can create another melted dataframe that contains the forward-looking 7-day returns. This will be our "target variable" for evaluating our trading strategy.

Simply shift the pivoted dataset by -7  to get "future" prices, like so:

In [56]:
# Try figure this step out on your own...

fdelta_dict = dict() # 'forward delta'
for offset in [7, 14, 21, 28]:
    fdelta_dict[f'fdelta_{offset}']= pivoted_df.shift(-offset)/pivoted_df-1 # forward looking change formula: future val/original val -1
    


{'fdelta_7': Code        GWA_BTC  GWA_ETH  GWA_LTC  GWA_XLM  GWA_XRP
 Date                                                   
 2014-04-01    -0.06      nan    -0.14      nan    -0.08
 2014-04-02    -0.02      nan    -0.06      nan     0.05
 2014-04-03    -0.07      nan    -0.04      nan     0.03
 2014-04-04    -0.09      nan    -0.07      nan    -0.10
 2014-04-05    -0.05      nan     0.01      nan    -0.13
 ...             ...      ...      ...      ...      ...
 2018-01-19      nan      nan      nan      nan      nan
 2018-01-20      nan      nan      nan      nan      nan
 2018-01-21      nan      nan      nan      nan      nan
 2018-01-22      nan      nan      nan      nan      nan
 2018-01-23      nan      nan      nan      nan      nan
 
 [1394 rows x 5 columns],
 'fdelta_14': Code        GWA_BTC  GWA_ETH  GWA_LTC  GWA_XLM  GWA_XRP
 Date                                                   
 2014-04-01     0.01      nan    -0.07      nan    -0.17
 2014-04-02     0.13      nan     0

In [ ]:
# // your code here
#return_df

display(fdelta_dict)

In [57]:
# Melt the return dataset and append to list
#  // your code here


for offset in offset_lst:
    melted_dfs.append(
    fdelta_dict[f'fdelta_{offset}'].reset_index().melt(id_vars= ['Date'], 
                                                       value_name = f"fdelta_{offset}")
    )

print(melted_dfs[-1])

           Date     Code  fdelta_28
0    2014-04-01  GWA_BTC      -0.08
1    2014-04-02  GWA_BTC      -0.03
2    2014-04-03  GWA_BTC       0.05
3    2014-04-04  GWA_BTC       0.02
4    2014-04-05  GWA_BTC      -0.02
...         ...      ...        ...
6965 2018-01-19  GWA_XRP        nan
6966 2018-01-20  GWA_XRP        nan
6967 2018-01-21  GWA_XRP        nan
6968 2018-01-22  GWA_XRP        nan
6969 2018-01-23  GWA_XRP        nan

[6970 rows x 3 columns]


## Reduce-merge the melted data
All that's left to do is join our melted dataframes into a single analytical base table. We'll need two tools.

The first is Pandas's merge function, which works like SQL JOIN. For example, to merge the first two melted dataframes...

In [58]:
# Merge two dataframes
pd.merge(melted_dfs[0], melted_dfs[1], on=['Date', 'Code']).tail()

,Date,Code,delta_7,delta_14
6965,2018-01-19,GWA_XRP,-0.22,-0.41
6966,2018-01-20,GWA_XRP,-0.29,-0.42
6967,2018-01-21,GWA_XRP,-0.30,-0.51
6968,2018-01-22,GWA_XRP,-0.32,-0.52
6969,2018-01-23,GWA_XRP,-0.04,-0.48


See how we now have delta_7 and delta_14 in the same row? This is the start of our analytical base table. All we need to do now is merge all of our melted dataframes together with a base dataframe of other features we might want.

The most elegant way to do this is using Python's built-in reduce function. First we'll need to import it:

In [60]:
from functools import reduce

In [61]:
# try do this step on your own so that you can have a ready ABT with following data dictionary:

# Date: The day on which the index values were calculated.
# Code: Which cryptocurrency.
# VWAP: The volume weighted average price traded that day.
# delta_7: Return over the prior 7 days (1.0 = 100% return).
# delta_14: Return over the prior 14 days (1.0 = 100% return).
# delta_21: Return over the prior 21 days (1.0 = 100% return).
# delta_28: Return over the prior 28 days (1.0 = 100% return).
# return_7: Future return over the next 7 days (1.0 = 100% return).

In [122]:
# Your code here

base_df = df[['Date','Code', 'VWAP', 'TWAP']]
feature_dfs = [base_df] + melted_dfs 

abt= reduce(lambda d0,d1: d0.merge(d1,  on=['Date', 'Code']) ,feature_dfs)
abt['return_7'] = abt['fdelta_7']

abt.drop(['fdelta_7', 'fdelta_14', 'fdelta_21', 'fdelta_28'] , axis=1, inplace=True)

abt

,Date,Code,VWAP,TWAP,delta_7,delta_14,delta_21,delta_28,return_7
0,2014-04-01,GWA_BTC,482.76,482.82,nan,nan,nan,nan,-0.06
1,2014-04-02,GWA_BTC,460.19,465.93,nan,nan,nan,nan,-0.02
2,2014-04-03,GWA_BTC,432.29,433.28,nan,nan,nan,nan,-0.07
3,2014-04-04,GWA_BTC,443.46,443.93,nan,nan,nan,nan,-0.09
4,2014-04-05,GWA_BTC,452.53,452.95,nan,nan,nan,nan,-0.05
...,...,...,...,...,...,...,...,...,...
6304,2018-01-19,GWA_XRP,1.82,1.81,-0.22,-0.41,0.02,0.65,nan
6305,2018-01-20,GWA_XRP,1.75,1.75,-0.29,-0.42,-0.26,0.53,nan
6306,2018-01-21,GWA_XRP,1.55,1.56,-0.30,-0.51,-0.28,0.45,nan
6307,2018-01-22,GWA_XRP,1.43,1.42,-0.32,-0.52,-0.35,0.30,nan


### Question: which cryptocurrency to pick based on the biggest momentum on a particular day (based on back looking delta)

For example, if we wanted to pick the coin that had the biggest momentum on Jan 23rd, 2018, we could simply display the rows for that date and look at the 7, 14, 21, and 28-day prior returns:

In [123]:
#  your code here
display(
    abt.loc[abt['Date']=='2018-01-23'],
    abt.loc[abt['Date']=='2018-01-23'].sort_values('delta_7'),
    
)

,Date,Code,VWAP,TWAP,delta_7,delta_14,delta_21,delta_28,return_7
1393,2018-01-23,GWA_BTC,"10,921.00","10,929.33",-0.11,-0.31,-0.25,-0.31,nan
2293,2018-01-23,GWA_ETH,992.05,992.91,-0.12,-0.22,0.13,0.29,nan
3687,2018-01-23,GWA_LTC,176.95,177.12,-0.13,-0.32,-0.29,-0.37,nan
4914,2018-01-23,GWA_XLM,0.47,0.47,-0.02,-0.24,-0.10,1.16,nan
6308,2018-01-23,GWA_XRP,1.42,1.42,-0.04,-0.48,-0.39,0.29,nan


,Date,Code,VWAP,TWAP,delta_7,delta_14,delta_21,delta_28,return_7
3687,2018-01-23,GWA_LTC,176.95,177.12,-0.13,-0.32,-0.29,-0.37,nan
2293,2018-01-23,GWA_ETH,992.05,992.91,-0.12,-0.22,0.13,0.29,nan
1393,2018-01-23,GWA_BTC,"10,921.00","10,929.33",-0.11,-0.31,-0.25,-0.31,nan
6308,2018-01-23,GWA_XRP,1.42,1.42,-0.04,-0.48,-0.39,0.29,nan
4914,2018-01-23,GWA_XLM,0.47,0.47,-0.02,-0.24,-0.10,1.16,nan


#### Aggregate with group-by
As a final step, if we wanted to only keep the first days of each month, we can use a group-by followed by an aggregation.

1. First, create a new 'month'  feature from the first 7 characters of the Date strings.
2. Then, group the observations by  'Code' and by  'month'. Pandas will create "cells" of data that separate observations by Code and month.
3. Finally, within each group, simply take the  .first() observation and reset the index.

Note: We're assuming your dataframe is still properly sorted by date.

In [126]:
# Your code here
# dir(abt.iloc[3687]['Date'])
# abt.iloc[3687]['Date'].__str__()[:7]

abt.reindex(abt['Date'])
abt.sort_index(inplace=True)


abt['Month'] = abt['Date'].apply(lambda s: s.__str__()[:7])

monthly_delta_df = pd.DataFrame(                                     
                                      abt.groupby(['Month','Code']).first().reset_index(),
)
                                      
#abt.groupby(['Month']).first().reset_index(),

display(
    type(monthly_delta_df),
    monthly_delta_df,
    monthly_delta_df.shape,
    )

pandas.core.frame.DataFrame

,Month,Code,Date,VWAP,TWAP,delta_7,delta_14,delta_21,delta_28,return_7
0,2014-04,GWA_BTC,2014-04-01,482.76,482.82,-0.06,0.01,0.03,-0.08,-0.06
1,2014-04,GWA_LTC,2014-04-01,13.19,13.20,-0.14,-0.07,-0.06,-0.22,-0.14
2,2014-04,GWA_XRP,2014-04-01,0.01,0.01,-0.08,-0.17,-0.28,-0.39,-0.08
3,2014-05,GWA_BTC,2014-05-01,455.13,455.10,-0.07,-0.10,0.13,0.05,-0.03
4,2014-05,GWA_LTC,2014-05-01,11.14,11.12,-0.10,-0.14,0.14,0.10,-0.05
...,...,...,...,...,...,...,...,...,...,...
204,2018-01,GWA_BTC,2018-01-01,"14,103.18","14,093.73",-0.01,-0.26,-0.16,0.23,0.13
205,2018-01,GWA_ETH,2018-01-01,765.22,765.15,0.02,0.04,0.64,0.64,0.57
206,2018-01,GWA_LTC,2018-01-01,227.89,227.91,-0.17,-0.29,0.28,1.28,0.17
207,2018-01,GWA_XLM,2018-01-01,0.45,0.44,0.99,0.68,2.31,3.87,0.45


(209, 10)

As you can see, we now have a proper ABT with:

Only relevant data from the 1st day of each month.
Momentum features calculated from the prior 7, 14, 21, and 28 days.
The future returns you would've made 7 days later.
In other words, we have exactly what we need to evaluate the simple trading strategy we proposed at the beginning!